# TensorFlow Deploy - TFHub 

Storing TensorFlow modules with `tfd` helps solve two problems. First of all, it allows to store and archive our own modules in a safe location, with no need of publishing them anywhere. Second of all, it allows to use internal modules within external company resources, once we clone world modules of our interest.

We will show you here how modules embedded in [tfhub.dev](tfhub.dev) can be moved to `tfd`. Next, we will build a model using the module.

In [ ]:
import tensorflow as tf
import tensorflow_deploy_utils as tfd
import tensorflow_hub as tfh
from helpers import grpc_text_client

In [ ]:
tf.__version__, tfd.__version__, tfh.__version__

Let's download a module from [tfhub.dev](tfhub.dev).

In [ ]:
# It may take a while for the first time, because module weights almost 200MB
tfhub_module = tfh.load("https://tfhub.dev/google/nnlm-en-dim50/2")

Let's see if the module works properly

In [ ]:
embed1 = tfhub_module(["this model is quite heavy"])
embed1

Now we will save the module so that we can send it to our `tfd`

In [ ]:
EXPORT_PATH = "/tmp/tfhub_module/"
tf.saved_model.save(tfhub_module, EXPORT_PATH)

In order to be able to send our module to local `tfd` repository, we need to add README.md file. Therefore, we will create a cursor that will provide us with such a possibility.

In [ ]:
tfd_cursor = tfd.TFD(team="google", project="textembedding", name="nnlmendim50", host="tfd")

In [ ]:
tfd_cursor.generate_module_readme(dst_path=EXPORT_PATH, 
                                  description="Module copy from https://tfhub.dev/google/nnlm-en-dim50/2")

In [ ]:
tfd_cursor.upload_module(EXPORT_PATH)

Ready. The module we have chosen is now in our local `tfd` repository. Let's double check if it really is there. 

In [ ]:
tfd_cursor.list_modules()

Perfect, it seems that everything is working propely. Let's now build a model that will be using our module. 

## Model with our own module

In [ ]:
class MyModel(tf.keras.Model):
    
    def __init__(self, module_url):
        super(MyModel, self).__init__()
        self.module = tfh.load(module_url)
    
    @tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.dtypes.string, name="x")])
    def call(self, x):
        return self.module(x)
    
    @tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.dtypes.string, name="x")])
    def serve(self, x):
        return self.call(x)

In [ ]:
my_model = MyModel("http://tfd:9500/v1/modules/google/textembedding/names/nnlmendim50/versions/1")

We have built a model that includes a module to embed the text. Let's test if it works.

In [ ]:
embed2 = my_model(["this model is quite heavy"])
embed2

It seems to be working. But is it working correctly? Let's compare the results of module embedding `embed1` with model's embedding `embed2`. If everything is OK, the results should be identical.

In [ ]:
embed1 == embed2

As expected, both vectors are identical. Let's implement our new model into a respective `tfs` instance and see how it works. 

In [ ]:
NEW_EXPORT_PATH = "/tmp/my_model/"
tf.saved_model.save(my_model, NEW_EXPORT_PATH)

In [ ]:
tfd_cursor = tfd.TFD(team="demo", project="hub", name="nnlmendim50", host="tfd")

In [ ]:
tfd_cursor.generate_model_readme(dst_path=NEW_EXPORT_PATH, 
                                 description="Model created on top of tfhub.dev module: https://tfhub.dev/google/nnlm-en-dim50/2")

In [ ]:
response = tfd_cursor.deploy_model(src_path=NEW_EXPORT_PATH)
print(response)

Great, we have got a new model in our production. Let's test it now.

In [ ]:
response = grpc_text_client(dataset=[["this model is quite heavy"]], host="tfs-demo-hub", port=8500, 
                            model_name="nnlmendim50", model_label="canary")

In [ ]:
response

Let's see again if the response from `tfs` is the same as in the initial module. 

In [ ]:
embed3 = response[0].outputs["output_1"].float_val
embed3

In [ ]:
embed1 == embed3

Huray! Everything is perfect :) We have built a model based on tfhub.dev. Next, we implemented and tested the new model in an almost production environment. 

This is all we prepared in this notebook. If you have not done so already, we encourage you to get familiar with the rest of our notebooks. At the same time, you are very welcome to create your own notebooks describing some new interesting ways of using **TensorFlow Deploy** for your own cases.

Enjoy :)